In [4]:
import os
import psycopg2
from psycopg2.extras import RealDictCursor
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding

In [5]:
# config.py
DB_CONFIG = {
    "dbname": "enc2db_lab",
    "user": "postgres",
    "password": "Password123",      # nếu Postgres có password thì điền vào
    "host": "192.168.99.9",
    "port": 5432,
}
# 32 bytes = 256-bit AES key (DEMO: hard-code, thực tế phải lưu an toàn)
AES_KEY = b"0123456789abcdef0123456789abcdef"

# ORE (toy) cho numeric: c = A * x_scaled + B
# A > 0 để bảo toàn thứ tự. A,B là bí mật, chỉ client biết.

ORE_A = 7919          # prime > 0
ORE_B = 123456789123  # offset bí mật
ORE_SCALE = 100       # scale cho 2 chữ số thập phân (18,2)

# AHE numeric (toy): c = x_scaled + AHE_K
AHE_K = 987654321  # secret offset
AHE_SCALE = 100    # dùng chung scale 2 decimals cho dễ

# MHE numeric (toy): c = x_scaled * MHE_K
MHE_K = 123457        # prime-ish secret
MHE_SCALE = 5       # dùng chung 2 decimals cho dễ

## Step 2: ENCDB (Sofware Only)
### Step 2.1: Encryption and Decryption Engine (on Client)
Using EAS Encryption Algorithm

In [6]:
# aes_engine.py
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
# from config import AES_KEY


FIXED_IV = b"\x00" * 16   # 👈 IV cố định cho demo equality


def aes_encrypt(plaintext: str) -> bytes:
    if plaintext is None:
        raise ValueError("plaintext is None")

    cipher = Cipher(algorithms.AES(AES_KEY), modes.CBC(FIXED_IV))
    encryptor = cipher.encryptor()

    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded = padder.update(plaintext.encode("utf-8")) + padder.finalize()

    ct = encryptor.update(padded) + encryptor.finalize()
    return ct   # 👈 không cần prepend IV nữa, vì IV cố định


def aes_decrypt(blob: bytes) -> str:
    if blob is None:
        raise ValueError("cipher blob is None")

    cipher = Cipher(algorithms.AES(AES_KEY), modes.CBC(FIXED_IV))
    decryptor = cipher.decryptor()
    padded = decryptor.update(blob) + decryptor.finalize()

    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    plain_bytes = unpadder.update(padded) + unpadder.finalize()
    return plain_bytes.decode("utf-8")


In [7]:
# ahe_numeric.py
from decimal import Decimal
def ahe_encrypt_numeric(x: float | Decimal) -> int:
    """
    Toy AHE:
      x_scaled = round(x * SCALE)
      c = x_scaled + AHE_K
    Server chỉ thấy c, không biết x.
    """
    if x is None:
        raise ValueError("ahe_encrypt_numeric: value is None")

    dx = Decimal(str(x))
    scaled = int(dx * AHE_SCALE)
    return scaled + AHE_K


def ahe_decrypt_single(c: int) -> float:
    """
    Giải mã 1 ciphertext đơn lẻ, chủ yếu để debug.
    """
    scaled = c - AHE_K
    return scaled / AHE_SCALE


def ahe_decrypt_sum(sum_cipher: int, n: int) -> float:
    """
    sum_cipher = sum(x_scaled + AHE_K) = sum(x_scaled) + n*AHE_K
    => sum(x) = (sum_cipher - n*AHE_K) / SCALE
    """
    sum_scaled = sum_cipher - n * AHE_K
    return sum_scaled / AHE_SCALE

In [8]:
# mhe_numeric.py
from decimal import Decimal
# from config import MHE_K, MHE_SCALE

def mhe_encrypt_numeric(x: float | Decimal) -> int:
    """
    Toy MHE:
      x_scaled = round(x * SCALE)
      c = x_scaled * MHE_K
    Server chỉ nhân ciphertext.
    """
    if x is None:
        raise ValueError("mhe_encrypt_numeric: value is None")

    dx = Decimal(str(x))
    scaled = int(dx * MHE_SCALE)
    return scaled * MHE_K

def mhe_decrypt_single(c: int | Decimal) -> float:
    """
    Giải mã 1 ciphertext MHE (toy):
      c = x_scaled * MHE_K
      x = (c / MHE_K) / MHE_SCALE
    """
    dc = Decimal(str(c))
    scaled = dc / MHE_K
    plain = scaled / MHE_SCALE
    return float(plain)

def mhe_decrypt_product(prod_cipher: int, n: int) -> float:
    """
    Với n phần tử:
      prod_cipher = Π (xi_scaled * MHE_K)
                  = (Π xi_scaled) * (MHE_K ** n)

      => Π xi = (Π xi_scaled) / (SCALE ** n)
              = (prod_cipher / (MHE_K ** n)) / (MHE_SCALE ** n)

    Ở đây ta trả về "product" theo đúng scale (cẩn thận overflow và độ lớn).
    Mục đích chính là minh họa homomorphic, không phải để dùng thực tế.
    """
    from decimal import Decimal, getcontext

    # tăng precision để tránh mất mát khi n lớn
    getcontext().prec = 50

    prod_dec = Decimal(prod_cipher)
    k_pow_n = Decimal(MHE_K) ** n
    scale_pow_n = Decimal(MHE_SCALE) ** n

    prod_scaled = prod_dec / k_pow_n
    prod_plain = prod_scaled / scale_pow_n

    return float(prod_plain)


In [9]:
# ore.py
from decimal import Decimal
def ore_encrypt_numeric(x: float | Decimal) -> int:
    """
    ORE cho numeric (18,2):
      - scale x lên integer: x_scaled = round(x * SCALE)
      - ciphertext: c = A * x_scaled + B
    Bởi vì A > 0, nên:
      x1 < x2  =>  x1_scaled < x2_scaled  =>  c1 < c2
    """
    if x is None:
        raise ValueError("ore_encrypt_numeric: value is None")

    # làm việc với Decimal cho ổn hơn
    dx = Decimal(str(x))
    scaled = int(dx * ORE_SCALE)

    c = ORE_A * scaled + ORE_B
    return c

# không cần decryption trong ORE

In [10]:
def get_conn():
    return psycopg2.connect(**DB_CONFIG)

In [25]:
def insert_row_enc_all(plain_text: str, plain_numeric: float):
    aes_ct = aes_encrypt(plain_text)
    ore_ct = ore_encrypt_numeric(plain_numeric)
    ahe_ct = ahe_encrypt_numeric(plain_numeric)
    mhe_ct = mhe_encrypt_numeric(plain_numeric)

    sql = """
        INSERT INTO tbl_enc_all
            (plain_text, plain_numeric,
             aes_cipher, ore_cipher, ahe_cipher, mhe_cipher)
        VALUES (%s, %s, %s, %s::ore_en, %s, %s)
    """
    print(f"sql: {sql.strip()}")
    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute(
                sql,
                (
                    plain_text,
                    Decimal(str(plain_numeric)),
                    psycopg2.Binary(aes_ct),
                    ore_ct,
                    ahe_ct,
                    mhe_ct,
                ),
            )
        conn.commit()

In [21]:
def introduce_and_delete_all_rows():
    print("Table demo: tbl_enc_all (plain_text, plain_numeric , aes_cipher [bytes], ore_cipher [numeric], ahe_cipher [numeric], mhe_cipher [numeric])")
    sql = "DELETE FROM tbl_enc_all;"

    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute(sql)
    conn.commit()

def demo1():
    print("========Demo 1: Insert 03 rows:=========")
    insert_row_enc_all("Alice secret", 2.0)
    print("row 1: plain_text = Alice secret, plain_numeric = 2.0; encrypt plain_text => aes_cipher, plain_numeric => ahe_cipher, mhe_cipher")
    
    insert_row_enc_all("Bob secret",   3.0)
    print("row 2: plain_text = Bob secret, plain_numeric = 3.0; encrypt plain_text => aes_cipher, plain_numeric => ahe_cipher, mhe_cipher")
    
    insert_row_enc_all("Charlie secret", 4.0)
    print("row 3: plain_text = Charlie secret, plain_numeric = 4.0; encrypt plain_text => aes_cipher, plain_numeric => ahe_cipher, mhe_cipher")

def demo2():
    print("===========Demo 2: Print 3 rows just inserted without decryption===========")
    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute("SELECT * FROM tbl_enc_all ORDER BY id;")
            index = 1
            for r in cur.fetchall():
                # print(r)
                print(f"===========Row {index}===============")
                print(f"plain_text: {r['plain_text']}, plain_numeric: {r['plain_numeric']}, aes_cipher: {r['aes_cipher']}, ore_cipher: {r['ore_cipher']}, ahe_cipher: {r['ahe_cipher']}, mhe_cipher: {r['mhe_cipher']}")
                print(f"aes_cipher: {r['aes_cipher']}, ore_cipher: {r['ore_cipher']}, ahe_cipher: {r['ahe_cipher']}, mhe_cipher: {r['mhe_cipher']}")
                print(f"aes_cipher_decrypt: {aes_decrypt(r['aes_cipher'])}, ore_cipher_decrypt: no need decryption, ahe_cipher_decrypt: {ahe_decrypt_single(r['ahe_cipher'])}, mhe_cipher_decrypt: {mhe_decrypt_single(r['mhe_cipher'])}")
                index += 1

    


In [26]:
def demo3():
    print("========== Query by AES equal (plain_text='Bob secret') ==")
    aes_ct = aes_encrypt("Bob secret")
    sql = """
        SELECT id, aes_cipher, plain_text, plain_numeric
        FROM tbl_enc_all
        WHERE udf_aes_eq(aes_cipher, %s::bytea)
    """
    print(f"sql: {sql.strip()} ")
    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(sql, (psycopg2.Binary(aes_ct),))
            for r in cur.fetchall():
                print(f"Found row: id={r['id']}, aes_cipher={r['aes_cipher']}, plain_text={r['plain_text']}, plain_numeric={r['plain_numeric']}")

In [32]:
def demo4():
    print("====== Query numeric > 2.0 via ORE (and index) ======")
    ore_ct = ore_encrypt_numeric(2.0)

    sql = """
        SELECT id, plain_text, plain_numeric, ore_cipher
        FROM tbl_enc_all
        WHERE ore_cipher > %s::ore_en
        ORDER BY ore_cipher;
    """
    print(f"sql: {sql.strip()} ")
    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(sql, (ore_ct,))
            for r in cur.fetchall():
                print(f"Found row: id={r['id']}, plain_text={r['plain_text']}, plain_numeric={r['plain_numeric']}, ore_cipher={r['ore_cipher']}")

In [33]:
def demo5():
    print("======= AHE sum over plain_numeric =======")
    sql = """
        SELECT ahe_sum(ahe_cipher) AS sum_cipher,
               COUNT(*)            AS n
        FROM tbl_enc_all;
    """
    print(f"sql: {sql.strip()} ")
    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(sql)
            row = cur.fetchone()

    sum_cipher = row["sum_cipher"]
    n = row["n"]

    sum_plain = ahe_decrypt_sum(sum_cipher, n)
    print(f"AHE: sum_cipher = {sum_cipher}, n = {n}, decrypted sum = {sum_plain}")

In [36]:
def demo6():
    print("== MHE product over plain_numeric ==")
    sql = """
        SELECT mhe_prod(mhe_cipher) AS prod_cipher,
               COUNT(*)             AS n
        FROM tbl_enc_all;
    """
    print(f"sql: {sql.strip()} ")
    with get_conn() as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(sql)
            row = cur.fetchone()

    prod_cipher = row["prod_cipher"]
    n = row["n"]

    prod_plain = mhe_decrypt_product(prod_cipher, n)
    print(f"MHE: prod_cipher = {prod_cipher}, n = {n}, decrypted product = {prod_plain}")

In [38]:
introduce_and_delete_all_rows()
demo1()


Table demo: tbl_enc_all (plain_text, plain_numeric , aes_cipher [bytes], ore_cipher [numeric], ahe_cipher [numeric], mhe_cipher [numeric])
========Demo 1: Insert 03 rows:=========
sql: INSERT INTO tbl_enc_all
            (plain_text, plain_numeric,
             aes_cipher, ore_cipher, ahe_cipher, mhe_cipher)
        VALUES (%s, %s, %s, %s::ore_en, %s, %s)
row 1: plain_text = Alice secret, plain_numeric = 2.0; encrypt plain_text => aes_cipher, plain_numeric => ahe_cipher, mhe_cipher
sql: INSERT INTO tbl_enc_all
            (plain_text, plain_numeric,
             aes_cipher, ore_cipher, ahe_cipher, mhe_cipher)
        VALUES (%s, %s, %s, %s::ore_en, %s, %s)
row 2: plain_text = Bob secret, plain_numeric = 3.0; encrypt plain_text => aes_cipher, plain_numeric => ahe_cipher, mhe_cipher
sql: INSERT INTO tbl_enc_all
            (plain_text, plain_numeric,
             aes_cipher, ore_cipher, ahe_cipher, mhe_cipher)
        VALUES (%s, %s, %s, %s::ore_en, %s, %s)
row 3: plain_text = Charlie se

In [39]:
demo2()
demo3()

===========Demo 2: Print 3 rows just inserted without decryption===========
===========Row 1===============
plain_text: Alice secret, plain_numeric: 2.00, aes_cipher: <memory at 0x000001C21E70EB00>, ore_cipher: 123458372923, ahe_cipher: 987654521, mhe_cipher: 1234570
aes_cipher: <memory at 0x000001C21E70EB00>, ore_cipher: 123458372923, ahe_cipher: 987654521, mhe_cipher: 1234570
aes_cipher_decrypt: Alice secret, ore_cipher_decrypt: no need decryption, ahe_cipher_decrypt: 2.0, mhe_cipher_decrypt: 2.0
===========Row 2===============
plain_text: Bob secret, plain_numeric: 3.00, aes_cipher: <memory at 0x000001C21E70EA40>, ore_cipher: 123459164823, ahe_cipher: 987654621, mhe_cipher: 1851855
aes_cipher: <memory at 0x000001C21E70EA40>, ore_cipher: 123459164823, ahe_cipher: 987654621, mhe_cipher: 1851855
aes_cipher_decrypt: Bob secret, ore_cipher_decrypt: no need decryption, ahe_cipher_decrypt: 3.0, mhe_cipher_decrypt: 3.0
===========Row 3===============
plain_text: Charlie secret, plain_numeri

In [40]:
demo4()
demo5()
demo6()

====== Query numeric > 2.0 via ORE (and index) ======
sql: SELECT id, plain_text, plain_numeric, ore_cipher
        FROM tbl_enc_all
        WHERE ore_cipher > %s::ore_en
        ORDER BY ore_cipher; 
Found row: id=44, plain_text=Bob secret, plain_numeric=3.00, ore_cipher=123459164823
Found row: id=45, plain_text=Charlie secret, plain_numeric=4.00, ore_cipher=123459956723
======= AHE sum over plain_numeric =======
sql: SELECT ahe_sum(ahe_cipher) AS sum_cipher,
               COUNT(*)            AS n
        FROM tbl_enc_all; 
AHE: sum_cipher = 2962963863, n = 3, decrypted sum = 9.0
== MHE product over plain_numeric ==
sql: SELECT mhe_prod(mhe_cipher) AS prod_cipher,
               COUNT(*)             AS n
        FROM tbl_enc_all; 
MHE: prod_cipher = 5645058059174979000, n = 3, decrypted product = 24.0
